In [10]:
import os
import sys
import dotenv
import json

from pathlib import Path
from typing import List

sys.path.append('.')

from models.persona_info import(
    InterviewState,
    UserInterview
)

from models.interview_info import(
    InterviewInfo,
    InterviewAgentMessage
)

from models.persona_info import (
    PersonaInfo
)

from prompts.interview_prompt import(
    INITIAL_INTERVIEW_PROMPT_REVERSE,
    JOB_EXTENSION_INTERVIEW_PROMPT
)

from agents.extraction_agents import(
    extract_initial_persona_info
)

from my_utils import (
    save_json,
    read_json,
    get_agent
)

CONVERSATIONS_DIR = Path('conversations')
DATA_DIR = Path('data')

# Load API key from .env file
dotenv.load_dotenv("../env")

# Check if we're good to go
if not os.getenv("MISTRAL_API_KEY"):
    print("❌ No MISTRAL_API_KEY found!")
    print("Create an env file with your API key")
else:
    print("✅ API key found, we're ready to roll")

✅ API key found, we're ready to roll


In [11]:
# Load jobs domains map data
filename = f"map_clusters_jobs_v4.json"
save_path = DATA_DIR / filename
jobs_map = read_json(save_path)

In [21]:
def respond_to_chat(chat_history: List[str], conversation_id) -> str:
    return ""

In [22]:
def get_agent_message_for_initial_interview(
    interview,
    model: str = "mistral-medium-latest",
    print_prompt = False
) -> InterviewAgentMessage:
    prompt = INITIAL_INTERVIEW_PROMPT_REVERSE
    interview_agent = get_agent(prompt, model_id=model)
    conversation_str = '\n'.join(interview)
    agent_response = interview_agent.structured_output(output_model=InterviewAgentMessage, prompt=conversation_str)
    return agent_response

In [23]:
def get_agent_message_for_job_extension_interview(
    interview,
    model: str = "mistral-medium-latest",
    print_prompt = False
) -> InterviewAgentMessage:
    domains_str = ""
    for domain in jobs_map:
        domains_str += f"  - {domain} : {jobs_map[domain]['description']}" + "\n"

    prompt = JOB_EXTENSION_INTERVIEW_PROMPT.format(
        domains_str=domains_str
    )

    if print_prompt is True:
        print(prompt)
    
    interview_agent = get_agent(prompt, model_id=model)
    conversation_str = '\n'.join(interview)
    agent_response = interview_agent.structured_output(output_model=InterviewAgentMessage, prompt=conversation_str)
    return agent_response

In [24]:
def parse_conversation_for_initial_interview(
    conversation,
    model: str = "mistral-medium-latest",  
    print_prompt = False
) -> PersonaInfo:
    persona_info = extract_initial_persona_info(
        conversation,
        model = model,
        print_prompt=print_prompt
    )
    return persona_info

In [25]:
def route_message_to_agent(
    userInterview,
    model: str = "mistral-medium-latest"    
) -> InterviewAgentMessage:
    if userInterview.interview_state == InterviewState.INITIAL_INTERVIEW:
        agent_response = get_agent_message_for_initial_interview(userInterview.interview, model=model)
    elif userInterview.interview_state == InterviewState.JOB_DOMAIN_INTERVIEW:
        agent_response = get_agent_message_for_job_extension_interview(userInterview.interview, model=model)
    else:
        print(f"interview_state not supported : {userInterview.interview_state}")
        return None
    return agent_response

In [26]:
def parse_conversation(
    userInterview
)-> UserInterview:
    if userInterview.interview_state == InterviewState.INITIAL_INTERVIEW:
        persona_info = parse_conversation_for_initial_interview(userInterview.interview, print_prompt = False)
        userInterview.persona_info = persona_info
        if persona_info.recommendation_type == 'jobs_trainings':
            userInterview.interview_state = InterviewState.JOB_DOMAIN_INTERVIEW
        elif persona_info.recommendation_type == 'trainings_only':
            userInterview.interview_state = InterviewState.TRAINING_DOMAIN_INTERVIEW
        else:
            userInterview.interview_state = InterviewState.OPEN_DISCUSSION
            
    return userInterview

In [27]:
userInterview = UserInterview()

interview = InterviewInfo()

conversation_id = 1

filename = f"user_interview_{conversation_id}.json"
user_interview_path = CONVERSATIONS_DIR / filename
if not user_interview_path.exists():
    userInterview_data = userInterview.model_dump_json()
    save_json(user_interview_path, userInterview_data)
else:
    userInterview_data = json.loads(read_json(user_interview_path))
    userInterview = UserInterview.model_validate(userInterview_data)

#print(userInterview)

#print(userInterview)

agent_response = get_agent_message_for_job_extension_interview(userInterview.interview, print_prompt=True)
print(agent_response)

while False:
    user_input = input("Type something (or 'exit' to quit): ")

    if user_input.lower() == 'exit':
        print("Goodbye!")
        break
    else:
        userInterview.interview.append(f"User: {user_input}")
        agent_response = route_message_to_agent(userInterview)
        print(agent_response)
        if agent_response.conversation_finished is False:
            agent_message = agent_response.message
            userInterview.interview.append(f"Assistant: {agent_message}")
            # print(userInterview)
            userInterview_data = userInterview.model_dump_json()
            save_json(user_interview_path, userInterview_data)
        else:
            userInterview_data = userInterview.model_dump_json()
            save_json(user_interview_path, userInterview_data)

            userInterview = parse_conversation(userInterview)
            userInterview_data = userInterview.model_dump_json()
            save_json(user_interview_path, userInterview_data)

            agent_response = route_message_to_agent(userInterview)


get_agent_message_for_job_extension_interview
  - Financial Operations And Compliance Management : Managing financial activities, ensuring regulatory compliance, and supporting risk management in finance.
  - Legal Research And Procedural Support : Conducting legal research, drafting documents, and supporting procedural compliance and case management.
  - Insurance Operations And Compliance Management : Managing insurance processes, ensuring compliance, assessing risks, and handling client services.
  - Maritime And Port Operations Management : Managing and coordinating vessel, terminal, and cargo operations in maritime and port environments.
  - Industrial Production And Live Event Technical Operations : Encompasses equipment design, maintenance, and coordination in industrial and live event settings.
  - Health Safety And Environmental Compliance : Ensuring workplace safety, regulatory compliance, and environmental health in industrial settings.
  - Procurement And Supplier Managemen

/tmp/ipykernel_23956/1566714797.py:21: DeprecationWarning: Agent.structured_output method is deprecated. You should pass in `structured_output_model` directly into the agent invocation. see: https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/structured-output/
  agent_response = interview_agent.structured_output(output_model=InterviewAgentMessage, prompt=conversation_str)


conversation_finished=False message='Great, Rio is a vibrant city with many opportunities. To start, could you tell me which domains you are interested in? Here are some options: Financial Operations and Compliance Management, Legal Research and Procedural Support, Insurance Operations and Compliance Management, Maritime and Port Operations Management, Industrial Production and Live Event Technical Operations, Health Safety and Environmental Compliance, Procurement and Supplier Management, Electrical and Electronics Systems Engineering, Community Engagement and Cultural Development, Tourism and Hospitality Guest Services, Visual Arts and Studio Production, Cultural Information and Records Management, Design Research and Innovation Management, Fiber and Paper Manufacturing Operations, Food Production Quality and Safety Management.'


In [ ]:

    selected_index = default_index
else:
    try:
        selected_index = int(user_input)
        if not (1 <= selected_index <= len(files)):
            raise ValueError
    except ValueError:
        print("Invalid input. Using default.")
        selected_index = default_index

selected_file = files[selected_index - 1]

prompt = f"Are you sure you want to proceed with '{selected_file}'? [y/N]: "
response = input(prompt).strip().lower()

if response != 'y':
    print("Canceled by user")
elif sanity_check():
    # Load results
    results_save_path = SUBMISSION_DIR / selected_file
    results = read_json(results_save_path)
    print("Results loaded")

    # Submit
    print("...Submitting results...")
    response = make_submission(results)
    print("Results submited")

    if response and response.status_code == 200:
        print("🎉 Submission successful! Check the leaderboard!")
        print(response)
    else:
        print(f"❌ Submission failed: {response.text if response else 'No response'}")

    # Final cost report
    print("\n" + "="*50)
    print_cost_summary()
else:
    print("Problem with data")

In [10]:
print(response)

None


In [5]:
import ipywidgets as widgets
from IPython.display import display

# Create text input and button
text_input = widgets.Text(
    value='',
    placeholder='Type something',
    description='Input:',
    disabled=False
)

submit_button = widgets.Button(
    description='Submit',
    button_style='success'
)

# Define the callback function
def on_button_click(b):
    print(f"You typed: {text_input.value}")

# Link the button to the callback
submit_button.on_click(on_button_click)

# Display the widgets
display(text_input, submit_button)

Text(value='', description='Input:', placeholder='Type something')

Button(button_style='success', description='Submit', style=ButtonStyle())